In [1]:
from utils import generate_real_samples, generate_latent_points, load_real_samples, generate_fake_samples, generate_images
from numpy import ones
import matplotlib.pyplot as plt
from statistics import mean
from wgan import WGAN
from tensorflow.keras.models import load_model

In [2]:
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_critic=5):
    # Calculate the number of batches per epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)
    # Calculate the total number of training steps
    n_steps = bat_per_epo * n_epochs
    # Calculate the size of half a batch
    half_batch = int(n_batch / 2)
    # Initialize lists to store the history of losses
    c1_hist, c2_hist, g_hist = list(), list(), list()
    # Loop over the total number of training steps
    for i in range(n_steps):
        # Initialize lists to store the losses for this step
        c1_tmp, c2_tmp = list(), list()
        # Train the critic for n_critic times
        for _ in range(n_critic):
            # Generate real samples and train the critic on them
            X_real, y_real = generate_real_samples(dataset, half_batch)
            c_loss1 = c_model.train_on_batch(X_real, y_real)
            c1_tmp.append(c_loss1)
            # Generate fake samples and train the critic on them
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            c_loss2 = c_model.train_on_batch(X_fake, y_fake)
            c2_tmp.append(c_loss2)
        # Store the average losses for this step
        c1_hist.append(mean(c1_tmp))
        c2_hist.append(mean(c2_tmp))
        # Generate points in the latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # Create labels for the fake samples
        y_gan = -ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        g_hist.append(g_loss)
        # Print the progress
        print('> %d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))

    plt.plot(c1_hist, label='crit_real')
    plt.plot(c2_hist, label='crit_fake')
    plt.plot(g_hist, label='gen')
    plt.legend()
    plt.savefig('plot_line_plot_loss.png')
    plt.close()

In [3]:
dataset = load_real_samples()
# Define the dimensionality of the latent space
latent_dim = 50
# Create a WGAN
wgan = WGAN(dataset.shape, latent_dim)
# Build the critic and the generator
discriminator = wgan.build_discriminator()
generator = wgan.build_generator()
# Build the combined model
wgan_model = wgan.build_wgan()

# Train the WGAN
train(g_model=generator, c_model=discriminator, gan_model=wgan_model, dataset=dataset, latent_dim=latent_dim)

c:\Users\user\anaconda\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
c:\Users\user\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 0s 71ms/step
> 1, c1=-2.574, c2=-0.021 g=-0.351
1/1 [==============================] - 0s 153ms/step
> 2, c1=-7.138, c2=-0.023 g=-1.715
1/1 [==============================] - 0s 162ms/step
> 3, c1=-9.827, c2=-0.030 g=-3.224
1/1 [==============================] - 0s 166ms/step
> 4, c1=-12.673, c2=-0.038 g=-4.396
1/1 [==============================] - 0s 82ms/step
> 5, c1=-14.753, c2=-0.045 g=-5.729
1/1 [==============================] - 0s 155ms/step
> 6, c1=-16.321, c2=-0.052 g=-7.067
1/1 [==============================] - 0s 114ms/step
> 7, c1=-17.874, c2=-0.072 g=-8.178
1/1 [==============================] - 0s 81ms/step
> 8, c1=-19.766, c2=-0.080 g=-9.194
1/1 [==============================] - 0s 156ms/step
> 9, c1=-20.291, c2=-0.104 g=-10.055
1/1 [==============================] - 0s 151ms/step
> 10, c1=-21.549, c2=-0.136 g=-10.871
1/1 [==============================] - 0s 106ms/step
> 11, c1=-22.320, c2=-0.161 g=-11.748
1/1 [=================